In [22]:
import os
import json
import boto3
import pandas as pd
import numpy as np
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer
from boto3.dynamodb.conditions import Key
from decimal import Decimal

In [23]:
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [25]:

ts= TypeSerializer()
td = TypeDeserializer()

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

In [26]:
def query_sql(beer_id, dynamodb=None):
    if not dynamodb:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))
    stmt = "SELECT BREW_ID,PI_TMSTP,STAGE,location,temperature,thermo_loc,thermo_temp FROM BREW_MON WHERE BREW_ID='" + beer_id + "'  ORDER BY PI_TMSTP DESC"
   # print(lambda_handler(event=brew1))
    response = dyna_clnt.execute_statement(Statement=stmt)
    print("HI")
    return response    
 

In [95]:
lines = query_sql(beer_id='SCWG_IPA')

HI


In [133]:
 df_raw = pd.json_normalize(lines, record_path =['Items'])

In [134]:
    df_raw['pi_time']=pd.to_datetime(df_raw['PI_TMSTP.S'])
    df_raw['temperature_n']=pd.to_numeric(df_raw['temperature.S'], downcast="float")
    df_raw['thermo_temp_n']=pd.to_numeric(df_raw['thermo_temp.N'], downcast="float")    
    df_raw.set_index('pi_time',inplace=True)

In [135]:
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n > 50].index)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n < 0].index)
    df_raw = df_raw.drop(columns=['BREW_ID.S','PI_TMSTP.S','STAGE.S','location.S','thermo_loc.S','temperature.S','thermo_temp.N'])
    df_raw['temperature_n'] = df_raw['temperature_n'].round(2)
    df_raw['thermo_temp_n'] = df_raw['thermo_temp_n'].round(2)

In [136]:
    df_4h = df_raw.groupby([pd.Grouper(freq='2H', level='pi_time')]).mean(numeric_only=True)
    df_4h = df_4h[df_4h['temperature_n'].notna()]
    df_4h = df_4h[df_4h['thermo_temp_n'].notna()]

In [119]:
print(df_4h)

                     temperature_n  thermo_temp_n
pi_time                                          
2021-09-08 00:00:00      13.250000      17.820000
2021-09-09 22:00:00      20.111111      19.073750
2021-09-10 00:00:00      18.500000      19.103333
2021-09-10 02:00:00      18.000000      19.052500
2021-09-10 04:00:00      17.250000      19.039999
...                            ...            ...
2021-09-17 08:00:00      16.916666      17.522499
2021-09-17 10:00:00      19.250000      17.477499
2021-09-17 12:00:00      20.333334      17.484999
2021-09-17 14:00:00      21.083334      17.633333
2021-09-17 16:00:00      21.250000      17.712500

[94 rows x 2 columns]


In [137]:
    df_4h=df_4h.reset_index()
  

In [138]:
  df_4h_json = df_4h.to_json(orient='records',date_format='iso')

In [122]:
    json_4h = {'Items': df_4h_json}

In [139]:
print(df_4h_json)


[{"pi_time":"2021-09-08T00:00:00.000","temperature_n":13.25,"thermo_temp_n":17.8199996948},{"pi_time":"2021-09-09T22:00:00.000","temperature_n":20.1111106873,"thermo_temp_n":19.0737495422},{"pi_time":"2021-09-10T00:00:00.000","temperature_n":18.5,"thermo_temp_n":19.1033325195},{"pi_time":"2021-09-10T02:00:00.000","temperature_n":18.0,"thermo_temp_n":19.0524997711},{"pi_time":"2021-09-10T04:00:00.000","temperature_n":17.25,"thermo_temp_n":19.0399990082},{"pi_time":"2021-09-10T06:00:00.000","temperature_n":17.1666660309,"thermo_temp_n":18.9750003815},{"pi_time":"2021-09-10T08:00:00.000","temperature_n":18.75,"thermo_temp_n":19.0249996185},{"pi_time":"2021-09-10T10:00:00.000","temperature_n":21.3333339691,"thermo_temp_n":18.9958324432},{"pi_time":"2021-09-10T12:00:00.000","temperature_n":21.75,"thermo_temp_n":19.0816669464},{"pi_time":"2021-09-10T14:00:00.000","temperature_n":22.5833339691,"thermo_temp_n":19.0949993134},{"pi_time":"2021-09-10T16:00:00.000","temperature_n":22.6666660309,"t

In [113]:

json = df_4h.to_json() 
  


In [140]:
print(df_4h_json)

[{"pi_time":"2021-09-08T00:00:00.000","temperature_n":13.25,"thermo_temp_n":17.8199996948},{"pi_time":"2021-09-09T22:00:00.000","temperature_n":20.1111106873,"thermo_temp_n":19.0737495422},{"pi_time":"2021-09-10T00:00:00.000","temperature_n":18.5,"thermo_temp_n":19.1033325195},{"pi_time":"2021-09-10T02:00:00.000","temperature_n":18.0,"thermo_temp_n":19.0524997711},{"pi_time":"2021-09-10T04:00:00.000","temperature_n":17.25,"thermo_temp_n":19.0399990082},{"pi_time":"2021-09-10T06:00:00.000","temperature_n":17.1666660309,"thermo_temp_n":18.9750003815},{"pi_time":"2021-09-10T08:00:00.000","temperature_n":18.75,"thermo_temp_n":19.0249996185},{"pi_time":"2021-09-10T10:00:00.000","temperature_n":21.3333339691,"thermo_temp_n":18.9958324432},{"pi_time":"2021-09-10T12:00:00.000","temperature_n":21.75,"thermo_temp_n":19.0816669464},{"pi_time":"2021-09-10T14:00:00.000","temperature_n":22.5833339691,"thermo_temp_n":19.0949993134},{"pi_time":"2021-09-10T16:00:00.000","temperature_n":22.6666660309,"t

In [68]:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))

In [69]:

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))



In [82]:
table = dynamodb.Table('BREW_LOG')

In [83]:
print(table.creation_date_time)

2020-06-21 17:34:00.674000+02:00


In [141]:
table.update_item(
   Key={"BEER_ID":"SCWG_IPA", "BEER_DATE": "2021-09-05"},
                UpdateExpression="set TEMP_DATA=:r",
                ExpressionAttributeValues={":r": df_4h_json},
                ReturnValues="UPDATED_NEW"
)

{'Attributes': {'TEMP_DATA': '[{"pi_time":"2021-09-08T00:00:00.000","temperature_n":13.25,"thermo_temp_n":17.8199996948},{"pi_time":"2021-09-09T22:00:00.000","temperature_n":20.1111106873,"thermo_temp_n":19.0737495422},{"pi_time":"2021-09-10T00:00:00.000","temperature_n":18.5,"thermo_temp_n":19.1033325195},{"pi_time":"2021-09-10T02:00:00.000","temperature_n":18.0,"thermo_temp_n":19.0524997711},{"pi_time":"2021-09-10T04:00:00.000","temperature_n":17.25,"thermo_temp_n":19.0399990082},{"pi_time":"2021-09-10T06:00:00.000","temperature_n":17.1666660309,"thermo_temp_n":18.9750003815},{"pi_time":"2021-09-10T08:00:00.000","temperature_n":18.75,"thermo_temp_n":19.0249996185},{"pi_time":"2021-09-10T10:00:00.000","temperature_n":21.3333339691,"thermo_temp_n":18.9958324432},{"pi_time":"2021-09-10T12:00:00.000","temperature_n":21.75,"thermo_temp_n":19.0816669464},{"pi_time":"2021-09-10T14:00:00.000","temperature_n":22.5833339691,"thermo_temp_n":19.0949993134},{"pi_time":"2021-09-10T16:00:00.000","t

In [144]:
nested_preferences = {
    "theme": "dark",
    "notifications": {
        "email": True,
        "sms": False
    },
    "settings": {
        "language": "en",
        "timezone": "GMT+0"
    }
}

# Example user item with nested preferences
user_item = {
'BEER_ID':"TESTSCWG_IPA", 'BEER_DATE': "2021-09-05",
    'email': 'john@example.com',
    'age': 30,
    'preferences': nested_preferences
}

# Get reference to the DynamoDB table
table = dynamodb.Table("BREW_LOG")

# Put the item into the table
table.put_item(Item=user_item)

{'ResponseMetadata': {'RequestId': '8GJ5DQSPGQ0QGKA175KEVVM56FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 18 Dec 2023 15:26:10 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8GJ5DQSPGQ0QGKA175KEVVM56FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}